# Twitter Review Dataset

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk 
import re 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

## Importing Dataset

In [2]:
ps = PorterStemmer()
df = pd.read_csv('tweet.csv')

In [3]:
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


## First review :

In [ ]:
text = df['tweet'][0]

## Cleaning first review :

In [7]:
text = re.sub('[^a-zA-Z]',' ',text)

In [8]:
text

'  user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction     run'

In [9]:
text = text.lower()
text = text.split()

In [10]:
text

['user',
 'when',
 'a',
 'father',
 'is',
 'dysfunctional',
 'and',
 'is',
 'so',
 'selfish',
 'he',
 'drags',
 'his',
 'kids',
 'into',
 'his',
 'dysfunction',
 'run']

In [11]:
text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]

In [12]:
text

['user', 'father', 'dysfunct', 'selfish', 'drag', 'kid', 'dysfunct', 'run']

In [13]:
text = ' '.join(text)

In [14]:
text

'user father dysfunct selfish drag kid dysfunct run'

# for all reviews

In [15]:
df.shape

(22275, 3)

In [18]:
df.iloc[:,1:3]

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation
...,...,...
22270,0,ascot times with this babe â¤ï¸â¤ï¸ #ascot...
22271,0,happy monday #positivity #monday
22272,1,you're running out of #hater bitches to #whit...
22273,0,@user @user babe wtf i thot we were gonna do t...


In [19]:
clean_review = []
for i in range(22275):
    txt = re.sub('[^a-zA-Z]',' ',df['tweet'][i])
    txt = txt.lower()
    txt = txt.split()
    txt = [ps.stem(word) for word in txt if not word in set(stopwords.words('english'))]
    txt = ' '.join(txt)
    clean_review.append(txt)

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=500)

In [21]:
X = cv.fit_transform(clean_review)

In [22]:
y = df['label'].values

# Spliting dataset into train test 

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [24]:
X_train.shape

(16706, 500)

In [25]:
X_test.shape

(5569, 500)

In [26]:
X.shape

(22275, 500)

# Applying LogisticRegression

In [27]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [28]:
clf = logreg.fit(X_train, y_train)

In [29]:
y_pred = clf.predict(X_test)

In [30]:
score = clf.score(X_test, y_test)

In [31]:
score

0.9455916681630454

   # Accuracy -- 94.559%  

# Applying Decision Tree 

In [34]:
from sklearn.tree import DecisionTreeClassifier
dtf = DecisionTreeClassifier()

In [35]:
clf1 = dtf.fit(X_train, y_train)

In [36]:
y_pred1 = clf1.predict(X_test)

In [37]:
score1 = clf1.score(X_test, y_test)

In [38]:
score1

0.9188364158735859

# Accuracy --- 91.88%